In [272]:
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import pickle
import numpy as np
from pathlib import Path
from enum import Enum
import scipy.constants as sc
from scipy.signal import savgol_filter
#from tensorboard.backend.event_processing.event_file_loader import EventFileLoader

#plt = matplotlib.pyplot

In [273]:
class Tag(Enum):
    CUMULATIVE_REWARD = "cumulative_rewards"
    EP_LENGTH = "ep_length"
    DOOR_PASSAGE = "passage"
    COLLISION_INITIAL = "initial"
    COLLISION_STAY = "stay"
    COLLISION_TOTAL = "total"
    DOOR_GOOD = "good_passage"
    DOOR_BAD = "bad_passage"
    TARGET_REACHED = "target_reached"

class Plot(Enum):
    COLLISION = 0
    ENV1 = 1
    ENV2 = 2
    LINEPLOT = 3
    RESULT_DATA_ENV1 = 4
    RESULT_DATA_ENV2 = 5
    DOOR = 6
    DOOR_HIST = 7
    SENSOR_STUDY_EP_LENGTH = 8
    SENSOR_STUDY_REWARD = 9
    SENSOR_STUDY_DOOR_GOOD = 10,
    SENSOR_STUDY_DOOR_BAD = 11,
    SENSOR_STUDY_INITIAL_COLLISION = 12
    SENSOR_STUDY_CONTINUED_COLLISION = 13
    SENSOR_STUDY_DOOR_QUALITY = 14
    RESULT_DATA_SENSOR_STUDY = 15
    STEP_PENALTY_EP_LENGTH = 16

tag_colors = {
    Tag.CUMULATIVE_REWARD: "C1",
    Tag.EP_LENGTH: "C2"
    }

Select from what run to get data. Also choose the Type of data.

In [274]:
multiple_runs = True
selected_runs: list[int] = [6465]#list(range(6466, 6475))# [6465]#list(range(6475, 6480)) # list(range(6466, 6475)) #[6072]6466 - 6474
plot_step = 5
plot_all = True

# Select the plot you want:
selected_plot = Plot.RESULT_DATA_ENV2

In [275]:
"""Get y data fro the summary dict using the ID of the training run.
The tag specifies the selected type of data."""
def get_y_data(summary_dict: dict, id: int, tag: Tag):
    try:
        if tag in [Tag.CUMULATIVE_REWARD, Tag.EP_LENGTH, Tag.TARGET_REACHED]:
            return summary_dict[id]["env"][tag.value]
        if tag in [Tag.DOOR_PASSAGE, Tag.DOOR_BAD, Tag.DOOR_GOOD]:
            return summary_dict[id]["door"][tag.value]
        if tag in [Tag.COLLISION_STAY, Tag.COLLISION_INITIAL, Tag.COLLISION_STAY]:
            return summary_dict[id]["collision"][tag.value]
    except ValueError:
        return []


In [276]:
"""Get x data from the summary dict. Is based on the recorded step."""
def get_x_data(summary_dict: dict, id: int, tag: Tag):
    d = len(get_y_data(summary_dict, id, tag))
    return summary_dict[id]["steps"][:d]

In [277]:
"""Filter data to smooth plots."""
def filter_data(data):
    try: 
        return savgol_filter(data, 15, 2)
    except ValueError:
        return []

In [278]:
def cut_arrays_to_same_length(a: np.ndarray, b:np.ndarray):
    if len(a) != len(b):
        if len(a) < len(b):
            d = len(a)
            b = b[:d]
        else:
            d = len(b)
            a = a[:d]
    return a, b

In [279]:
def get_passage_quality(good, bad):
    x = good
    y = bad
    try: 
        result = x / (x + y)
    except RuntimeWarning:
        result = 0.0
    return result

In [280]:
# Load the pickle with all the run data.
summary_file_path = Path("C:/Users/max.muehlefeldt/Documents/GitHub/unity-machine-learning/python/basic_rl_env/summary_dict.pickle").absolute()

with open(summary_file_path, mode="rb") as file:
    summary_dict = pickle.load(file)

In [281]:
# Global plot settings
matplotlib.rcParams['font.size'] = 15
matplotlib.rcParams["font.family"] = 'serif'

In [282]:
if selected_plot in [
    Plot.SENSOR_STUDY_EP_LENGTH, 
    Plot.SENSOR_STUDY_REWARD,
    Plot.SENSOR_STUDY_INITIAL_COLLISION,
    Plot.SENSOR_STUDY_CONTINUED_COLLISION,
    Plot.SENSOR_STUDY_DOOR_GOOD,
    Plot.SENSOR_STUDY_DOOR_BAD,
    Plot.SENSOR_STUDY_DOOR_QUALITY
    ]:
    selected_tag = Tag.CUMULATIVE_REWARD
    if selected_plot is Plot.SENSOR_STUDY_EP_LENGTH:
        selected_tag = Tag.EP_LENGTH
    elif selected_plot is Plot.SENSOR_STUDY_INITIAL_COLLISION:
        selected_tag = Tag.COLLISION_INITIAL
    elif selected_plot is Plot.SENSOR_STUDY_CONTINUED_COLLISION:
        selected_tag = Tag.COLLISION_STAY
    elif selected_plot is Plot.SENSOR_STUDY_DOOR_GOOD:
        selected_tag = Tag.DOOR_GOOD
    elif selected_plot is Plot.SENSOR_STUDY_DOOR_BAD:
        selected_tag = Tag.DOOR_BAD
    elif selected_plot is Plot.SENSOR_STUDY_DOOR_QUALITY:
        selected_tag = Tag.DOOR_BAD

    fig = plt.figure(figsize=(9, 7))
    fig.patch.set_alpha(0.)
    gs = fig.add_gridspec(ncols=1, nrows=1, figure=fig)
    axs_first = fig.add_subplot(gs[0, 0])

    for id in selected_runs:
        #sensor_count = content[id]['stats']['sensorCount']
        sensor_count = summary_dict[id]["file_contents"]["unity_config"]["sensorCount"]
        #print(sensor_count)
        #run_label = f"{sensor_count} sensor" if sensor_count == 1 else f"{sensor_count} sensors, run {id}"
        run_label = f"{sensor_count} sensor" if sensor_count == 1 else f"{sensor_count} sensors"
        #run_label += f", no LSTM" if id == 6108 else ", with LSTM"

        
        #x = content[id]["x_values"][selected_tag]

        x = get_x_data(summary_dict, id, selected_tag)
        #y = savgol_filter( content[id]["y_values"][selected_tag], 15, 2)
        
        if selected_plot is Plot.SENSOR_STUDY_DOOR_QUALITY:
            selected_tag = Tag.DOOR_BAD
            y_bad = -1 * np.array(get_y_data(summary_dict, id, selected_tag))
            selected_tag = Tag.DOOR_GOOD
            y_good = np.array(get_y_data(summary_dict, id, selected_tag))
            y_bad, y_good = cut_arrays_to_same_length(y_bad, y_good)
            #y = []
            #for index in range(len(y_good)):
                #y.append(get_passage_quality(y_good[index], y_bad[index]))
            y = get_passage_quality(y_good, y_bad)
        elif selected_plot is Plot.SENSOR_STUDY_CONTINUED_COLLISION:
            y = filter_data(filter_data(filter_data(y)))
        elif selected_plot is Plot.SENSOR_STUDY_DOOR_BAD:
            y = -1 * filter_data(get_y_data(summary_dict, id, selected_tag))
        else:
            y = filter_data(get_y_data(summary_dict, id, selected_tag))
        #x = content[id]["x_values"][selected_tag]
        
        axs_first.plot(
            x, 
            y,
            #color=tag_colors[selected_tag]
            label=run_label
            )
    
    
    if selected_plot is Plot.SENSOR_STUDY_REWARD:
        axs_first.set_ylim(-2, 1.6)
        axs_first.axhline(1.5, label="Reward limit = 1.5", color="C3", linestyle="dashed")
        #axs_first.axhline(1.25, label="Reward limit = 1.25", color="C3", linestyle="dashed")
        #axs_first.axhline(1.0, label="Reward limit = 1.0", color="C3", linestyle="dashed")
        axs_first.set_ylabel("Reward")
        axs_first.set_title("Cumulative reward")

    elif selected_plot is Plot.SENSOR_STUDY_EP_LENGTH:
        axs_first.set_ylim(0, 200)
        axs_first.set_ylabel("Length")
        axs_first.set_title("Episode length")
    
    elif selected_plot is Plot.SENSOR_STUDY_INITIAL_COLLISION:
        axs_first.set_ylim(0, 200)
        axs_first.set_ylabel("Count")
        axs_first.set_title("Initial Collisions")

    elif selected_plot is Plot.SENSOR_STUDY_CONTINUED_COLLISION:
        axs_first.set_ylim(0, 200)
        axs_first.set_ylabel("Count")
        axs_first.set_title("Continued Collisions")
    
    elif selected_plot is Plot.SENSOR_STUDY_DOOR_GOOD:
        axs_first.set_title("Correct Door Passages")
        axs_first.set_ylim(0, 95)

    elif selected_plot is Plot.SENSOR_STUDY_DOOR_BAD:
        axs_first.set_title("Incorrect Door Passages")
        axs_first.set_ylim(0, 95)
        
    axs_first.set_xlabel("Step")
    
    axs_first.legend()
    axs_first.grid()   

    plt.subplots_adjust(hspace=0.4)
    plt.style.use("default")
    plt.tight_layout()
    #plt.savefig(f"{selected_run}_all.pdf")
    fig.savefig(f"{selected_runs[0]}-{selected_runs[-1]}_{selected_plot}.pdf")


In [283]:
if selected_plot is Plot.COLLISION:
    fig = plt.figure(figsize=(9, 7))
    fig.patch.set_alpha(0.)
    gs = fig.add_gridspec(ncols=1, nrows=1, figure=fig)
    axs_first = fig.add_subplot(gs[0, 0])

    for id in selected_runs:
        #sensor_count = content[id]['stats']['sensorCount']
        sensor_count = summary_dict[id]["file_contents"]["unity_config"]["sensorCount"]
        #print(sensor_count)
        #run_label = f"{sensor_count} sensor" if sensor_count == 1 else f"{sensor_count} sensors, run {id}"
        run_label = f"{sensor_count} sensor" if sensor_count == 1 else f"{sensor_count} sensors"
        #run_label += f", no LSTM" if id == 6108 else ", with LSTM"

        #x = content[id]["x_values"][selected_tag]
        x = get_x_data(summary_dict, id, selected_tag)
        #y = savgol_filter( content[id]["y_values"][selected_tag], 15, 2)
        #y = savgol_filter( get_y_data(summary_dict, id, selected_tag), 15, 2)
        #x = content[id]["x_values"][selected_tag]
        #axs_first.plot(
        #    x, 
        #    y,
        #    #color=tag_colors[selected_tag]
        #    label=run_label
        #    )

        
        y_initial = filter_data(get_y_data(summary_dict, id, Tag.COLLISION_INITIAL))
        y_stay = filter_data(get_y_data(summary_dict, id, Tag.COLLISION_STAY))

        if len(y_initial) != len(y_stay):
            if len(y_initial) < len(y_stay):
                d = len(y_initial)
                y_stay = y_stay[:d]
                x = x[:d]
            else:
                d = len(y_stay)
                y_initial = y_initial[:d]
                x = x[:d]
        
        y = np.vstack([y_initial, y_stay])

        

        axs_first.stackplot(
            x, y,
            labels=["Initial contact", "Continued contact"],
            alpha=0.8
        )

        axs_first.set_ylim(0, 200)
    #if selected_tag is Tag.CUMULATIVE_REWARD:
    #    #axs_first.set_ylim(-2, 1.5)
    #    axs_first.set_ylim(-2, 1.1)
        #axs_first.axhline(1.25, label="Reward limit = 1.25", color="C3", linestyle="dashed")
    #    axs_first.axhline(1.0, label="Reward limit = 1.0", color="C3", linestyle="dashed")
        axs_first.set_ylabel("Count")
        axs_first.set_title("Collisions")

    #elif selected_tag is Tag.EP_LENGTH:
    #    axs_first.set_ylim(0, 200)
    #    axs_first.set_ylabel("Length")
    #    axs_first.set_title("Episode length")
    
    #elif selected_tag is Tag.DOOR_PASSAGE:
    #    axs_first.set_ylim(-1, 1)
    #    axs_first.set_ylabel("Passage Quality")
    #    axs_first.set_title("Door passage")
    
        axs_first.set_xlabel("Step")
        
        axs_first.legend(title=f"Run {id}")
        axs_first.grid()   

        plt.subplots_adjust(hspace=0.4)
        plt.style.use("default")
        plt.tight_layout()
        #plt.savefig(f"{selected_run}_all.pdf")
        fig.savefig(f"{selected_runs[0]}-{selected_runs[-1]}_{selected_plot}.pdf")

       

In [284]:
if selected_plot in [Plot.ENV1, Plot.ENV2]:
    fig = plt.figure(figsize=(9, 7))
    #fig = plt.figure()
    fig.patch.set_alpha(0.)
    
    gs = fig.add_gridspec(ncols=1, nrows=1, figure=fig)

    # First plot.
    #selected_tag = Tag.EP_LENGTH
    axs_first = fig.add_subplot(gs[0, 0])
    #fig.suptitle(f"Runs {selected_runs[0]} - {selected_runs[-1]}", y=0.95)
    #fig.tight_layout()

    for id in selected_runs:
        #sensor_count = content[id]['stats']['sensorCount']
        sensor_count = summary_dict[id]["file_contents"]["unity_config"]["sensorCount"]
        #print(sensor_count)
        #run_label = f"{sensor_count} sensor" if sensor_count == 1 else f"{sensor_count} sensors, run {id}"

        run_label = f"Cumulutive reward"
        selected_tag = Tag.CUMULATIVE_REWARD
        #x = content[id]["x_values"][selected_tag]
        x = get_x_data(summary_dict, id, selected_tag)
        #y = savgol_filter( content[id]["y_values"][selected_tag], 15, 2)
        y = filter_data(get_y_data(summary_dict, id, selected_tag))
        #x = content[id]["x_values"][selected_tag]
        axs_first.plot(
            x, 
            y,
            color=tag_colors[selected_tag],
            label="Cumulative reward",
            linewidth=2.0,
            )
        axs_first.set_ylim(
            -2, 
            1.1 if selected_plot is Plot.ENV1 else 1.6
        )
        axs_first.set_ylabel("Reward")
        reward_label = "Reward limit = 1.0" if selected_plot is Plot.ENV1 else  "Reward limit = 1.5"
        axs_first.axhline(
            1.0 if selected_plot is Plot.ENV1 else 1.5,
            label=reward_label, color="C3", linestyle="dashed"
        )

        selected_tag = Tag.EP_LENGTH
        x = get_x_data(summary_dict, id, selected_tag)
        y = filter_data(get_y_data(summary_dict, id, selected_tag))
        axs_second = axs_first.twinx()
        axs_second.plot(
            x, 
            y,
            color=tag_colors[selected_tag],
            label="Episode length",
            linewidth=2.0,
            )
        axs_second.set_ylim(0, 200)
        axs_second.set_ylabel("Episode Length")
        #fig.suptitle("Cumulative reward and episode length")
    
    """if selected_tag is Tag.CUMULATIVE_REWARD:
        #axs_first.set_ylim(-2, 1.5)
        axs_first.set_ylim(-2, 1.1)
        #axs_first.axhline(1.25, label="Reward limit = 1.25", color="C3", linestyle="dashed")
        axs_first.axhline(1.0, label="Reward limit = 1.0", color="C3", linestyle="dashed")
        axs_first.set_ylabel("Reward")
        axs_first.set_title("Cumulative reward")

    elif selected_tag is Tag.EP_LENGTH:
        axs_first.set_ylim(0, 200)
        axs_first.set_ylabel("Length")
    axs_first.set_title("Episode length")"""
    axs_first.set_title("Cumulative reward and episode length")
    axs_first.set_xlabel("Step")
    
    axs_second.legend()
    axs_first.legend(title=f"Run {id}")
    axs_second.grid()
    #axs_first.grid()

    plt.subplots_adjust(hspace=0.4)
    plt.style.use("default")
    plt.tight_layout()
    #plt.savefig(f"{selected_run}_all.pdf")
    fig.savefig(f"{selected_runs[0]}-{selected_runs[-1]}_{selected_plot}.pdf")

In [285]:
if selected_plot is Plot.DOOR:
    fig = plt.figure(figsize=(9, 7))
    #fig = plt.figure()
    fig.patch.set_alpha(0.)
    
    gs = fig.add_gridspec(ncols=1, nrows=1, figure=fig)

    # First plot.
    #selected_tag = Tag.EP_LENGTH
    axs_first = fig.add_subplot(gs[0, 0])
    #fig.suptitle(f"Runs {selected_runs[0]} - {selected_runs[-1]}", y=0.95)
    #fig.tight_layout()

    for id in selected_runs:
              
        selected_tag = Tag.DOOR_BAD
        x = get_x_data(summary_dict, id, selected_tag)
        y = filter_data(get_y_data(summary_dict, id, selected_tag))
        y = -1 * y

        axs_first.plot(
            x, 
            y,
            #color=tag_colors[selected_tag],
            label="Incorrect passage",
            linewidth=2.0,
            )

        selected_tag = Tag.DOOR_GOOD
        x = get_x_data(summary_dict, id, selected_tag)
        y = filter_data(get_y_data(summary_dict, id, selected_tag))
       
        axs_first.plot(
            x, 
            y,
            label="Correct passage",
            linewidth=2.0,
            )
        
    axs_first.set_title("Door passage")
    axs_first.set_xlabel("Step")
    
    
    axs_first.legend(title=f"Run {id}")
    axs_first.grid()

    plt.subplots_adjust(hspace=0.4)
    plt.style.use("default")
    plt.tight_layout()
    #plt.savefig(f"{selected_run}_all.pdf")
    fig.savefig(f"{selected_runs[0]}-{selected_runs[-1]}_{selected_plot}.pdf")

In [286]:
if selected_plot is Plot.DOOR_HIST:
    fig = plt.figure(figsize=(9, 7))
    #fig = plt.figure()
    fig.patch.set_alpha(0.)
    
    gs = fig.add_gridspec(ncols=1, nrows=1, figure=fig)

    # First plot.
    #selected_tag = Tag.EP_LENGTH
    axs_first = fig.add_subplot(gs[0, 0])
    #fig.suptitle(f"Runs {selected_runs[0]} - {selected_runs[-1]}", y=0.95)
    #fig.tight_layout()

    for id in selected_runs:
              
        selected_tag = Tag.DOOR_BAD
        x = get_x_data(summary_dict, id, selected_tag)
        y_bad = -1 * filter_data(get_y_data(summary_dict, id, selected_tag))
        #y = -1 * y


        #axs_first.hist(
        #    #x, 
        #    y,
        #    #color=tag_colors[selected_tag],
        #    label="Incorrect passage",
        #    linewidth=2.0,
        #    )

        selected_tag = Tag.DOOR_GOOD
        x = get_x_data(summary_dict, id, selected_tag)
        y_good = filter_data(get_y_data(summary_dict, id, selected_tag))

        y_bad, y_good = cut_arrays_to_same_length(y_bad, y_good)

        # Use a constant bin width to make the two histograms easier to compare visually
        #bin_width = 1
        #bins = np.arange(np.min([y_bad, y_good]),
                            #np.max([y_bad, y_good]) + bin_width, bin_width)
        axs_first.hist(
            #x, 
            y_bad,
            weights=-np.ones_like(y_bad),
            #bins=bins,
            label="Inorrect passage",
            )
        axs_first.hist(
            #x, 
            y_good,
            #bins=bins,
            label="Correct passage",
            )
        
    axs_first.set_title("Door passage")
    axs_first.set_xlabel("Step")
    axs_first.axhline(0, color="k")
    
    axs_first.legend()
    axs_first.grid()

    plt.subplots_adjust(hspace=0.4)
    plt.style.use("default")
    plt.tight_layout()
    #plt.savefig(f"{selected_run}_all.pdf")
    fig.savefig(f"{selected_runs[0]}-{selected_runs[-1]}_{selected_plot}.pdf")

In [287]:
if selected_plot in [
    Plot.STEP_PENALTY_EP_LENGTH
    ]:
    selected_tag = Tag.CUMULATIVE_REWARD

    fig = plt.figure(figsize=(9, 7))
    fig.patch.set_alpha(0.)
    gs = fig.add_gridspec(ncols=1, nrows=1, figure=fig)
    axs_first = fig.add_subplot(gs[0, 0])

    for id in selected_runs:
        step_penalty = summary_dict[id]["file_contents"]["unity_config"]["stepPenalty"]
        run_label = f"{step_penalty}"

        x = get_x_data(summary_dict, id, selected_tag)
        y = filter_data(get_y_data(summary_dict, id, selected_tag))
        
        axs_first.plot(
            x, 
            y,
            label=run_label
            )

    #axs_first.set_ylim(0, 200)
    axs_first.set_ylabel("Length")
    axs_first.set_title("Episode length")
    axs_first.set_xlabel("Step")
    
    axs_first.legend()
    axs_first.grid()   

    plt.subplots_adjust(hspace=0.4)
    plt.style.use("default")
    plt.tight_layout()
    #plt.savefig(f"{selected_run}_all.pdf")
    fig.savefig(f"{selected_runs[0]}-{selected_runs[-1]}_{selected_plot}.pdf")

## Get result data
To highlight the final performance of the NN.

In [288]:
def get_base_result_string(summary_dict: dict, id:int) -> str:

    # Initialise the string.
    s = ""

    # Get y data.
    y_collision_initial = get_y_data(summary_dict, id, Tag.COLLISION_INITIAL)
    y_collision_stay = get_y_data(summary_dict, id, Tag.COLLISION_STAY)
    y_cumulative_reward = get_y_data(summary_dict, id, Tag.CUMULATIVE_REWARD)
    y_ep_length = get_y_data(summary_dict, id, Tag.EP_LENGTH)
    y_target_reached = get_y_data(summary_dict, id, Tag.TARGET_REACHED)

    # Get x data.
    x_steps = get_x_data(summary_dict, id, Tag.EP_LENGTH)

    # Get other data.
    sensor_count = summary_dict[id]["file_contents"]["unity_config"]["sensorCount"]

    num_datapoints = 100

    s += "\\hline \n"
    s += "Parameter & Value & Performance Indicator & Value \\\\ \n"
    s += "\\hline \n"

    final_cumulative_reward = np.mean(y_cumulative_reward[-num_datapoints:])
    s+= f"Range sensor count & {sensor_count} & Cumulative reward & {np.round(final_cumulative_reward, 3)} \\\\ \n"

    final_cumulative_reward_std = np.std(y_cumulative_reward[-num_datapoints:])
    s+= f"Hidden layers & 1 & Cumulative reward STD & {np.round(final_cumulative_reward_std, 3)} \\\\ \n"

    final_ep_length = np.mean(y_ep_length[-num_datapoints:])
    s += f"Hidden units & 512 & Episode length & {np.round(final_ep_length, 3)} \\\\ \n"
    
    target_rate = (np.mean(y_target_reached[-num_datapoints:]) * 100) / (10000 / np.mean(y_ep_length[-num_datapoints:]))
    
    #final_collision_rate = 100 - (np.mean(y_initial[-10:]) / 10000) * 100
    steps_without_continued_col = 10000 - np.mean(y_collision_stay[-num_datapoints:])
    final_collision_rate = (np.mean(y_collision_initial[-num_datapoints:]) / (steps_without_continued_col)) * 100
    #print(100 -  / 10000 * 100)
    s += f"LSTM layer size & 64 & Target rate & {np.round(target_rate, 3)} \% \\\\ \n"

    s += f"& & Collision rate & {np.round(final_collision_rate, 3)}\% \\\\ \n"
    
    performed_steps = summary_dict[selected_runs[0]]["steps"][-1]
    performed_steps = "{:.0e}".format(performed_steps)
    s += f"& & Total performed steps & {performed_steps} \\\\ \n"
    s += "\\hline"

    return s


Get Env 1 data. 6459 is an applicable run.

In [289]:
if selected_plot is Plot.RESULT_DATA_ENV1:
    for id in selected_runs:
        print(get_base_result_string(summary_dict, id))


In [290]:
def get_passage_quality(x, y):
    return x / (x + y)

In [291]:
def get_env2_result_string(summary_dict: dict, id: int) -> str:
    s = get_base_result_string(summary_dict, id)
    s = "\\hline".join(s.split("\\hline")[:-1])

    # Get y data.
    y_door_passage_incorrect = get_y_data(summary_dict, id, Tag.DOOR_BAD)
    y_door_passage_correct = get_y_data(summary_dict, id, Tag.DOOR_GOOD)

    # Get x data.
    x_steps = get_x_data(summary_dict, id, Tag.EP_LENGTH)

    num_datapoints = 100

    door_passage_quality = get_passage_quality(
            np.mean(y_door_passage_correct[-num_datapoints:]),
            (-1* np.mean(y_door_passage_incorrect[-num_datapoints:])))
    s += f"& & Door passage quality & {np.round(door_passage_quality, 3)} \\\\ \n"
    s += "\\hline"

    return s
    

Generate environment 2 result table.

In [292]:
if selected_plot is Plot.RESULT_DATA_ENV2:
    for id in selected_runs:
        """
        y_collision_initial = get_y_data(summary_dict, id, Tag.COLLISION_INITIAL)
        y_collision_stay = get_y_data(summary_dict, id, Tag.COLLISION_STAY)
        y_cumulative_reward = get_y_data(summary_dict, id, Tag.CUMULATIVE_REWARD)
        y_ep_length = get_y_data(summary_dict, id, Tag.EP_LENGTH)
        y_door_passage_incorrect = get_y_data(summary_dict, id, Tag.DOOR_BAD)
        y_door_passage_correct = get_y_data(summary_dict, id, Tag.DOOR_GOOD)
        
        x_steps = get_x_data(summary_dict, id, Tag.EP_LENGTH)

        num_datapoints = 100
        """
        # Episode length.
        #final_cumulative_reward = np.mean(y_cumulative_reward[-num_datapoints:])
        #final_cumulative_reward = summary_dict[selected_runs[0]]["final_mean_reward"]
        #print(f"Cumulative reward & {np.round(final_cumulative_reward, 3)} \\\\")

        #final_cumulative_reward_std = summary_dict[selected_runs[0]]["final_std_reward"]
        #print(f"Cumulative reward STD & {np.round(final_cumulative_reward_std, 3)} \\\\")

        #final_ep_length = np.mean(y_ep_length[-10:])
        #print(f"Episode length & {np.round(final_ep_length, 2)} \\\\")

        #final_collision_rate = 100 - (np.mean(y_initial[-10:]) / 10000) * 100
        #final_collision_rate = (np.mean(y_collision_initial[-10:]) / 10000) * 100
        #print(100 -  / 10000 * 100)
        #print(f"Collision rate & {np.round(final_collision_rate, 3)}\% \\\\")

        #performed_steps = summary_dict[selected_runs[0]]["steps"][-1]
        #performed_steps = "{:.0e}".format(performed_steps)
        #print(f"Performed steps & {performed_steps} \\\\")

        
        """
        num_datapoints = 100
        print("\\hline")
        print("Parameter & Value & Performance Indicator & Value \\\\")
        print("\\hline")
        sensor_count = summary_dict[id]["file_contents"]["unity_config"]["sensorCount"]
        # Episode length.
        final_cumulative_reward = np.mean(y_cumulative_reward[-num_datapoints:])
        #final_cumulative_reward = summary_dict[selected_runs[0]]["final_mean_reward"]
        print(f"Range sensor count & {sensor_count} & Cumulative reward & {np.round(final_cumulative_reward, 3)} \\\\")

        final_cumulative_reward_std = np.std(y_cumulative_reward[-num_datapoints:])
        #final_cumulative_reward_std = summary_dict[selected_runs[0]]["final_std_reward"]
        print(f"Hidden layers & 1 & Cumulative reward STD & {np.round(final_cumulative_reward_std, 3)} \\\\")

        final_ep_length = np.mean(y_ep_length[-num_datapoints:])
        print(f"Hidden units & 512 & Episode length & {np.round(final_ep_length, 3)} \\\\")

        #final_collision_rate = 100 - (np.mean(y_initial[-10:]) / 10000) * 100
        steps_without_continued_col = 10000 - np.mean(y_collision_stay[-num_datapoints:])
        final_collision_rate = (np.mean(y_collision_initial[-num_datapoints:]) / (steps_without_continued_col)) * 100
        #print(100 -  / 10000 * 100)
        print(f"LSTM layer size & 64 & Collision rate & {np.round(final_collision_rate, 3)}\% \\\\")

        performed_steps = summary_dict[selected_runs[0]]["steps"][-1]
        performed_steps = "{:.0e}".format(performed_steps)
        print(f"& & Total performed steps & {performed_steps} \\\\")

        door_passage_quality = get_passage_quality(
            np.mean(y_door_passage_correct[-num_datapoints:]),
            (-1* np.mean(y_door_passage_incorrect[-num_datapoints:])))
        print(f"& & Door passage quality & {np.round(door_passage_quality, 3)} \\\\")
        print("\\hline")
        """
        print(get_env2_result_string(summary_dict, id))





\hline 
Parameter & Value & Performance Indicator & Value \\ 
\hline 
Range sensor count & 32 & Cumulative reward & 0.822 \\ 
Hidden layers & 1 & Cumulative reward STD & 0.019 \\ 
Hidden units & 512 & Episode length & 63.684 \\ 
LSTM layer size & 64 & Target rate & 96.271 \% \\ 
& & Collision rate & 0.085\% \\ 
& & Total performed steps & 1e+08 \\ 
& & Door passage quality & 0.802 \\ 
\hline


Generate Latex table containing sensor study results.

In [293]:
def calculate_position(x, min_val, max_val):
    # Calculate the midpoint
    midpoint = (min_val + max_val) / 2.0
    
    # Calculate the position
    position = 2 * (x - midpoint) / (max_val - min_val)
    
    return position

def get_colour_string(value, data: dict, tag, low_is_good = False) -> str:
    #tag = "final_ep_length"
    #value = 99.36
    #low_is_good = True

    all_values = []
    for id in data.keys():
        all_values.append(data[id][tag])

    """
    if low_is_good:
        all_values_sorted = np.flip(np.sort(all_values))
    else: # Normal case: High value equals good.
        all_values_sorted = np.sort(all_values)

    index = np.where(all_values_sorted == value)[0][0]

    colours = [
        "{red!90}",
        "{red!75}",
        "{red!50}",
        "{red!25}",
        "{yellow!50}",
        "{green!25}",
        "{green!50}",
        "{green!75}",
        "{green!90}",
    ]

    return f"\\cellcolor{colours[index]}"
    """

    max = np.max(all_values)
    min = np.min(all_values)

    pos = calculate_position(value, min, max)

    max_alpha_value = 80

    if low_is_good:
        if pos <= 0:
            s = f"{{green!{np.round(pos * -max_alpha_value, 2)}}}"
        else:
            s = f"{{red!{np.round(pos * max_alpha_value, 2)}}}"
    else:
        if pos <= 0:
            s = f"{{red!{np.round(pos * -max_alpha_value, 2)}}}"
        else:
            s = f"{{green!{np.round(pos * max_alpha_value, 2)}}}"

    return f"\\cellcolor{s}"

In [294]:
if selected_plot is Plot.RESULT_DATA_SENSOR_STUDY:
    run_data = {}

    for id in selected_runs:
        run_data[id] = {}
        y_collision_initial = get_y_data(summary_dict, id, Tag.COLLISION_INITIAL)
        y_collision_stay = get_y_data(summary_dict, id, Tag.COLLISION_STAY)
        y_cumulative_reward = get_y_data(summary_dict, id, Tag.CUMULATIVE_REWARD)
        y_ep_length = get_y_data(summary_dict, id, Tag.EP_LENGTH)
        y_door_passage_incorrect = get_y_data(summary_dict, id, Tag.DOOR_BAD)
        y_door_passage_correct = get_y_data(summary_dict, id, Tag.DOOR_GOOD)

        x_steps = get_x_data(summary_dict, id, Tag.EP_LENGTH)
        num_datapoints = 100
        
        # Episode length.
        #final_cumulative_reward = np.mean(y_cumulative_reward[-10:])
        final_cumulative_reward = summary_dict[id]["final_mean_reward"]
        #print(f"Cumulative reward & {np.round(final_cumulative_reward, 3)} \\\\")
        run_data[id]["final_cumulative_reward"] = np.round(final_cumulative_reward, 3)

        final_cumulative_reward_std = summary_dict[id]["final_std_reward"]
        #print(f"Cumulative reward STD & {np.round(final_cumulative_reward_std, 3)} \\\\")
        run_data[id]["final_cumulative_reward_std"] = np.round(final_cumulative_reward_std, 3)

        final_ep_length = np.mean(y_ep_length[-10:])
        #print(f"Episode length & {np.round(final_ep_length, 2)} \\\\")
        run_data[id]["final_ep_length"] = np.round(np.round(final_ep_length, 2), 3)

        #final_collision_rate = 100 - (np.mean(y_initial[-10:]) / 10000) * 100
        final_collision_rate = (np.mean(y_collision_initial[-10:]) / 10000) * 100
        #print(100 -  / 10000 * 100)
        #print(f"Collision rate & {np.round(final_collision_rate, 3)}\% \\\\")
        run_data[id]["final_collision_rate"] = np.round(final_collision_rate, 3)

        performed_steps = summary_dict[selected_runs[0]]["steps"][-1]
        #print(f"Performed steps & {performed_steps} \\\\")
        run_data[id]["performed_steps"] = performed_steps

        door_passage_quality = get_passage_quality(
            np.mean(y_door_passage_correct[-10:]),
            (-1* np.mean(y_door_passage_incorrect[-10:])))
        #print(f"Door passage quality $[-1, 1]$ & {np.round(door_passage_quality, 3)} \\\\")
        run_data[id]["door_passage_quality"] = np.round(door_passage_quality, 3)
        run_data[id]["door_passage_incorrect"] = -1 * np.mean(y_door_passage_incorrect[-10:])
        run_data[id]["door_passage_correct"] = np.mean(y_door_passage_correct[-10:])
        run_data[id]["door_passage_total"] = run_data[id]["door_passage_incorrect"] + run_data[id]["door_passage_correct"]
        
        run_data[id]["sensor_count"] = summary_dict[id]["file_contents"]['stats']['sensorCount']

    print("\\hline")
    print("& \\multicolumn{9}{c}{Sensor count} \\\\")
    
    s = f"Performance Indicator & "
    for id in selected_runs:
        s+= f"{run_data[id]['sensor_count']} & "
    s = s[:-2] + "\\\\"
    print(s)
    print("\\hline")

    colour_cell = True
    low_good = True
    high_good = False

    performance_indicators = [
        ("Sensors", "sensor_count"),
        ("Cumulative reward", "final_cumulative_reward", high_good),
        ("Cumulative reward STD", "final_cumulative_reward_std", low_good),
        ("Episode length", "final_ep_length", low_good),
        ("Collision rate", "final_collision_rate", low_good),
        ("Door passage quality","door_passage_quality", high_good),
        #("Incorrect Door passages","door_passage_incorrect"),
        #("Correct Door passages","door_passage_correct")
        ("Door passage count","door_passage_total", high_good)
    ]
    for indicator in performance_indicators:
        if indicator == ("Sensors", "sensor_count"):
            continue
        s = f"{indicator[0]} & "

        for id in run_data.keys():
            value = run_data[id][indicator[1]]
            s += f"{get_colour_string(value, run_data, indicator[1], low_is_good=indicator[2])}{value} & "
        s = s[:-2] + "\\\\"
        print(s)
    print("\\hline")

In [295]:
max = 5
min = 1
middle = max - min
value = 1

calculate_position(value, min, max)

-1.0

In [296]:
-1.0 * 100

-100.0